In [1]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
from influxdb_client.client.exceptions import InfluxDBError
import time
import numpy as np
import pandas as pd

/Users/maddy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [139]:
!pip install python-dotenv

import os
from dotenv import load_dotenv

# Wczytaj zmienne z pliku .env (upewnij się, że .env jest w tym samym folderze co notebook)
load_dotenv()

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


True

In [140]:
bucket = "redstone"
org = "redstone"
token = os.getenv('token')
# Store the URL of your InfluxDB instance
url=os.getenv('url')

In [141]:
client = InfluxDBClient(url=url, token=token, org=org, timeout=30000)

In [142]:
query_api = client.query_api()

## Extract last 30 days of USDC feeds

In [143]:
query = '''
from(bucket: "redstone")
  |> range(start: -30d)
  |> filter(fn: (r) => r._measurement == "dataPackages")
  |> filter(fn: (r) => r["dataServiceId"] == "redstone-main-demo")
  |> filter(fn: (r) => r["dataFeedId"] == "USDC")
'''

tables = query_api.query(query)

records = []
for table in tables:
    for record in table.records:
        records.append({
            "time": record.get_time(),
            "feed_name": record.values.get("_field"),
            "value": record.get_value()
        })

df = pd.DataFrame(records)

In [6]:
df.groupby('feed_name')['value'].describe().reset_index()

,feed_name,count,mean,std,min,25%,50%,75%,max
0,metadataValue,43199.0,0.999938,0.000062,0.999587,0.999901,0.999942,0.999980,1.000150
1,value,43199.0,0.999938,0.000062,0.999587,0.999901,0.999943,0.999980,1.000150
2,value-balancer-ethereum-usdt,43037.0,0.999916,0.000130,0.999251,0.999849,0.999913,0.999999,1.001450
3,value-binance-usdt,43171.0,0.999945,0.000082,0.999500,0.999895,0.999950,1.000000,1.000200
4,value-bitget-usdt,43184.0,0.999939,0.000085,0.999500,0.999890,0.999940,1.000000,1.000240
5,value-bitmart-usdt,43149.0,0.999955,0.000083,0.998050,0.999915,0.999960,1.000005,1.002014
6,value-bitrue-usdt,43174.0,0.999986,0.000172,0.999150,0.999900,0.999970,1.000026,1.000727
7,value-bitstamp-usd,43180.0,0.999978,0.000058,0.999360,0.999950,0.999980,1.000000,1.000780
8,value-coingecko,42369.0,0.999947,0.000046,0.999587,0.999903,0.999955,0.999992,1.000000
9,value-curve-ethereum-dai,43037.0,0.999961,0.000099,0.999473,0.999894,0.999967,1.000015,1.000473


### Remove value & metadataValue -> looks like this is a median already

In [7]:
df = df[(df['feed_name'] != 'metadataValue') & (df['feed_name'] != 'value')]

### Split feeds & slippage

In [8]:
is_slippage = df['feed_name'].str.startswith('value-slippage-')
df_values   = df[~is_slippage].copy()
df_slippage = df[ is_slippage].copy()

## Quick analysis

In [33]:
df_values.groupby('feed_name')['value'].agg(['min', 'max']).reset_index().sort_values('min')

,feed_name,min,max
9,value-kaiko-v2,0.989057,1.003951
12,value-lbank-usdt,0.997070,1.000235
3,value-bitmart-usdt,0.998050,1.002014
4,value-bitrue-usdt,0.999150,1.000727
0,value-balancer-ethereum-usdt,0.999251,1.001450
5,value-bitstamp-usd,0.999360,1.000780
11,value-kucoin-usdt,0.999435,1.000400
14,value-uniswap-v3-ethereum-dai-100,0.999454,1.000453
7,value-curve-ethereum-dai,0.999473,1.000473
1,value-binance-usdt,0.999500,1.000200


**value-coingecko** is intresting, because it's pricve never go above 1.0 and also the lowest value is at the same time the clostest to 1 from all feeds

**value-kaiko-v2** lowest price is 0.989057 which is the lowest from all (No assumtions yet -> it is a single point yet)

## Feed analysis

In [9]:
# Pivot df
feeds = df_values.pivot(index='time', columns='feed_name', values='value')
feeds.sort_index(inplace=True)
feeds.columns.name = None
feeds = feeds.reset_index()

In [10]:
assert len(feeds['time'].unique()) == len(feeds)

### Mising values

In [11]:
null_counts = feeds.isna().sum() \
            .rename('null_count') \
            .sort_values(ascending=False).reset_index()

null_counts['pct_of_total'] = (
    null_counts['null_count']
    .div(len(feeds))
    .mul(100)
    .round(2)
    .astype(str) + '%'
)

In [12]:
null_counts

,index,null_count,pct_of_total
0,value-kaiko-v2,9395,21.75%
1,value-lbank-usdt,1492,3.45%
2,value-coingecko,831,1.92%
3,value-curve-ethereum-usdt,181,0.42%
4,value-balancer-ethereum-usdt,163,0.38%
5,value-curve-ethereum-dai,163,0.38%
6,value-uniswap-v3-ethereum-dai-100,162,0.38%
7,value-bitmart-usdt,51,0.12%
8,value-binance-usdt,29,0.07%
9,value-kucoin-usdt,27,0.06%


At this stage we can see that **value-kaiko-v2** has big data gaps (21,78%) in single month.
**value-lbank-usdt** & **value-coingecko** feeds have higher gaps than most of the group, but it is acceptable

### Continue analysis on full df 
-> gaps will be taken into account as an anomaly later on

# Calc Standard Deviation for each feed

In [13]:
stability = feeds.set_index('time').std()
stability = stability.sort_values(ascending=False)

print("Feed stability (std deviation of price):")
print(stability)

Feed stability (std deviation of price):
value-kaiko-v2                       0.000455
value-bitrue-usdt                    0.000172
value-kucoin-usdt                    0.000131
value-balancer-ethereum-usdt         0.000130
value-lbank-usdt                     0.000118
value-curve-ethereum-dai             0.000099
value-okx-usdt                       0.000097
value-uniswap-v3-ethereum-dai-100    0.000097
value-curve-ethereum-usdt            0.000090
value-bitget-usdt                    0.000085
value-bitmart-usdt                   0.000083
value-binance-usdt                   0.000082
value-kraken-usdt                    0.000079
value-bitstamp-usd                   0.000058
value-coingecko                      0.000046
dtype: float64


value-kaiko-v2 is by far the most volatile (std = 0.000456), while value-coingecko (0.000045) and value-bitstamp-usd (0.000058) are the most stable, with the others falling in between.

Standard deviation is easy to compute and ranks feeds by overall variability, but it can be skewed by a few large spikes, doesn’t reveal when volatility occurs, and ignores issues like missing data or reporting delays.

In [14]:
feeds['value-global-median'] = feeds.drop('time', axis=1).median(axis=1)

## Outliers - base on IQR

In [16]:
df = feeds.set_index('time')
feed_cols = [c for c in df.columns if c != 'value-global-median']
df_feeds = df[feed_cols]

# 2. Compute quartiles and IQR per feed
Q1 = df_feeds.quantile(0.25)
Q3 = df_feeds.quantile(0.75)
IQR = Q3 - Q1

# 3. Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 4. Flag outliers
outliers = (df_feeds.lt(lower_bound, axis=1)) | (df_feeds.gt(upper_bound, axis=1))

# 5. Summarize counts and ratios
outlier_counts = outliers.sum().sort_values(ascending=False)
outlier_ratio  = (outlier_counts / len(df_feeds) * 100).round(2).sort_values(ascending=False)

print("\nOutlier ratio per feed (%):")
print(outlier_ratio)


Outlier ratio per feed (%):
value-kaiko-v2                       12.43
value-bitrue-usdt                    10.94
value-bitstamp-usd                    7.50
value-balancer-ethereum-usdt          3.73
value-bitmart-usdt                    2.95
value-lbank-usdt                      2.90
value-uniswap-v3-ethereum-dai-100     2.12
value-curve-ethereum-dai              1.74
value-kraken-usdt                     1.64
value-bitget-usdt                     1.42
value-binance-usdt                    1.39
value-curve-ethereum-usdt             1.15
value-okx-usdt                        0.71
value-kucoin-usdt                     0.26
value-coingecko                       0.12
dtype: float64


value-kaiko-v2 shows the highest outlier rate at 12.4%, followed by value-bitrue-usdt (10.9%) and value-bitstamp-usd (7.5%), indicating these feeds produce extreme price readings far more often than the rest. 

The most reliable sources are value-coingecko (0.12%) and value-kucoin-usdt (0.26%), which almost never report outliers, but is it good? -> probably not if there is a dpeg (the price might be delayed or over smoothed)

Weakness: The IQR-based outlier metric assumes a roughly symmetric distribution and may miss systematic biases or directional “worst-case” deviations (e.g., only large drops) that fall within the IQR bounds.

## Flatness (feed's median movements agains global median)

In [18]:
import pandas as pd

# assume `feeds` is your pivoted DataFrame, with 'time' as a column
df = feeds.set_index('time')

# separate out the feed columns vs. the global median column
feed_cols = [c for c in df.columns if c != 'value-global-median']
df_feeds = df[feed_cols]
median = df['value-global-median']

# compute first differences (t → t+1) and drop the NaN at top
feed_diffs = df_feeds.diff().dropna()
median_diff = median.diff().dropna()

# compute standard deviation of those diffs
feed_diff_std   = feed_diffs.std()
median_diff_std = median_diff.std()

# flatness ratio: how many times the market moves more than the feed
flatness_ratio = median_diff_std / feed_diff_std

# flatness_pct: percent below market volatility 
flatness_pct = (1 - feed_diff_std / median_diff_std) * 100

# combine into a single table
flatness = pd.DataFrame({
    'flatness_ratio': flatness_ratio,
    'flatness_%': flatness_pct.round(2)
}).sort_values('flatness_ratio', ascending=False)

print(flatness)

                                   flatness_ratio  flatness_%
value-bitstamp-usd                       2.560868       60.95
value-coingecko                          1.988871       49.72
value-curve-ethereum-usdt                1.056379        5.34
value-balancer-ethereum-usdt             0.978762       -2.17
value-uniswap-v3-ethereum-dai-100        0.851949      -17.38
value-curve-ethereum-dai                 0.851204      -17.48
value-kraken-usdt                        0.785880      -27.25
value-lbank-usdt                         0.782737      -27.76
value-bitmart-usdt                       0.640945      -56.02
value-okx-usdt                           0.548658      -82.26
value-kucoin-usdt                        0.547396      -82.68
value-bitget-usdt                        0.544310      -83.72
value-binance-usdt                       0.536414      -86.42
value-bitrue-usdt                        0.505233      -97.93
value-kaiko-v2                           0.075014    -1233.08


The “flatness” metric compares each feed’s short-term volatility (std of its price changes) against the global median.
Any extreme in this matric migh be a red flag. 
**High flatness_ratio** (value-bitstamp-usd & value-coingecko) might means that the feed is artificaly smotheed or it is slow to reflect real fluctuations.

**Low flatness_ratio** (value-kaiko-v2 & value-bitrue-usdt) might means the feed’s reported price moves more strongly than the actual USDC market moves. 

## Feed's delay against global median

In [106]:
import pandas as pd
import numpy as np

df = feeds.set_index('time')
feed_cols = [c for c in df.columns if c != 'value-global-median']

TH = 0.0002  # próg istotnego ruchu

# W ilu kolejnych interwałach sprawdzać "nadgonienie"
MAX_DELAY = 2

# Słownik na liczbę przypadków dla każdego feedu i delay
delay_results = {f'delayed_{d}_steps': {} for d in range(1, MAX_DELAY+1)}
delay_results['unresolved'] = {}
median_moves = df['value-global-median'].diff()
sig_move_idx = median_moves.abs() > TH
total_sig_moves = sig_move_idx.sum()
print(f"Total significant median movements above TH: {total_sig_moves}\n")

for feed in feed_cols:
    median_moves = df['value-global-median'].diff()
    feed_moves = df[feed].diff()
    sig_move_idx = median_moves.abs() > TH
    
    delayed = ~((feed_moves.abs() > TH) & sig_move_idx)
    
    # Sprawdzamy kolejne opóźnienia
    found_catchup = pd.Series(False, index=df.index)
    remaining = delayed & sig_move_idx
    
    for d in range(1, MAX_DELAY+1):
        # Czy feed nadrabia w d-tym kroku po ruchu mediany?
        catchup = (feed_moves.abs().shift(-d, fill_value=0) > TH) & remaining
        delay_results[f'delayed_{d}_steps'][feed] = catchup.sum()
        # Zaznaczamy już złapane, by nie liczyć kilka razy tego samego eventu
        found_catchup = found_catchup | catchup
        remaining = remaining & ~catchup
    delay_results['unresolved'][feed] = remaining.sum()
    

# Wyniki jako DataFrame
delay_df = pd.DataFrame(delay_results)
delay_df = delay_df[[f'delayed_{d}_steps' for d in range(1, MAX_DELAY+1)] + ['unresolved']]
delay_df = delay_df.sort_values(by='delayed_1_steps', ascending=False)
print(delay_df)


Total significant median movements above TH: 113

                                   delayed_1_steps  delayed_2_steps  \
value-bitrue-usdt                                9                2   
value-kaiko-v2                                   7                1   
value-kucoin-usdt                                7                2   
value-bitget-usdt                                5                0   
value-kraken-usdt                                5                1   
value-okx-usdt                                   5                0   
value-balancer-ethereum-usdt                     4                1   
value-binance-usdt                               4                2   
value-curve-ethereum-usdt                        3                2   
value-lbank-usdt                                 3                2   
value-bitmart-usdt                               2                1   
value-curve-ethereum-dai                         2                1   
value-uniswap-v3-ethereum-d

This metric highlights which feeds are slow to reflect real price changes.

Feeds like **value-bitstamp-usd** and **value-coingecko** have zero delayed reactions—but also the highest number of unresolved events, meaning they failed to react to any of the 113 significant USDC market moves. In contrast, other feeds, despite some delays, do eventually reflect most price changes, making them more responsive to real market movements.

## Median abs deviation

In [121]:
mean_abs_deviation = (df[feed_cols].sub(df['value-global-median'], axis=0)).abs().mean().sort_values(ascending=False)

In [122]:
mean_abs_deviation

value-kaiko-v2                       0.000173
value-kucoin-usdt                    0.000093
value-bitrue-usdt                    0.000089
value-balancer-ethereum-usdt         0.000075
value-lbank-usdt                     0.000074
value-okx-usdt                       0.000062
value-curve-ethereum-dai             0.000061
value-bitstamp-usd                   0.000059
value-uniswap-v3-ethereum-dai-100    0.000058
value-kraken-usdt                    0.000048
value-bitget-usdt                    0.000046
value-binance-usdt                   0.000046
value-coingecko                      0.000045
value-bitmart-usdt                   0.000040
value-curve-ethereum-usdt            0.000039
dtype: float64

### Corellation against global median

In [47]:
correlations = df[feed_cols].corrwith(df['value-global-median']).sort_values(ascending=False)
print("Correlation with global median per feed:")
print(correlations)

Correlation with global median per feed:
value-curve-ethereum-usdt            0.842741
value-bitmart-usdt                   0.695541
value-bitget-usdt                    0.686047
value-binance-usdt                   0.670268
value-balancer-ethereum-usdt         0.648620
value-kraken-usdt                    0.622492
value-curve-ethereum-dai             0.593108
value-uniswap-v3-ethereum-dai-100    0.583362
value-okx-usdt                       0.554838
value-kucoin-usdt                    0.504620
value-bitrue-usdt                    0.494104
value-lbank-usdt                     0.454369
value-coingecko                      0.397027
value-bitstamp-usd                   0.285659
value-kaiko-v2                       0.209167
dtype: float64


This metric quickly identifies which feeds reliably follow the collective market signal.

Feeds like **value-curve-ethereum-usdt** (correlation 0.84) and **value-bitmart-usdt** (0.70) move most closely in sync with the global median, showing strong alignment with the consensus price. Lower correlations, such as for **value-bitstamp-usd** (0.29) and **value-kaiko-v2** (0.21), suggest these feeds often diverge from broader market movements.

### Critical threshold

In [118]:
critical_threshold = 0.995
extreme_drop_count = (df[feed_cols] < critical_threshold).sum().sort_values(ascending=False)
print(f"Count of extreme drops below {critical_threshold} per feed:")
print(extreme_drop_count)

Count of extreme drops below 0.995 per feed:
value-kaiko-v2                       4
value-balancer-ethereum-usdt         0
value-binance-usdt                   0
value-bitget-usdt                    0
value-bitmart-usdt                   0
value-bitrue-usdt                    0
value-bitstamp-usd                   0
value-coingecko                      0
value-curve-ethereum-dai             0
value-curve-ethereum-usdt            0
value-kraken-usdt                    0
value-kucoin-usdt                    0
value-lbank-usdt                     0
value-okx-usdt                       0
value-uniswap-v3-ethereum-dai-100    0
dtype: int64


This metric highlights which feeds have ever reported dangerous depeg events, even if rare.

Only **value-kaiko-v2** reported any extreme drops below $0.995 4 times, while all other feeds never fell below this critical threshold during the analysis period—indicating they consistently tracked the USDC peg even in potentially volatile moments.

In [136]:
critical_upper_threshold = 1.003
extreme_up_count = (df[feed_cols] > critical_upper_threshold).sum().sort_values(ascending=False)
print(f"Count of extreme move above{critical_upper_threshold} per feed:")
print(extreme_up_count)

Count of extreme move above1.003 per feed:
value-kaiko-v2                       54
value-balancer-ethereum-usdt          0
value-binance-usdt                    0
value-bitget-usdt                     0
value-bitmart-usdt                    0
value-bitrue-usdt                     0
value-bitstamp-usd                    0
value-coingecko                       0
value-curve-ethereum-dai              0
value-curve-ethereum-usdt             0
value-kraken-usdt                     0
value-kucoin-usdt                     0
value-lbank-usdt                      0
value-okx-usdt                        0
value-uniswap-v3-ethereum-dai-100     0
dtype: int64


In [107]:
df_slippage

,time,feed_name,value
678770,2025-04-18 09:33:00+00:00,value-slippage-balancer-ethereum-usdt-buy,0.010862
678771,2025-04-18 09:34:00+00:00,value-slippage-balancer-ethereum-usdt-buy,0.010862
678772,2025-04-18 09:35:00+00:00,value-slippage-balancer-ethereum-usdt-buy,0.010862
678773,2025-04-18 09:36:00+00:00,value-slippage-balancer-ethereum-usdt-buy,0.010861
678774,2025-04-18 09:37:00+00:00,value-slippage-balancer-ethereum-usdt-buy,0.010862
...,...,...,...
1023027,2025-05-18 09:28:00+00:00,value-slippage-uniswap-v3-ethereum-dai-100-sell,-0.000091
1023028,2025-05-18 09:29:00+00:00,value-slippage-uniswap-v3-ethereum-dai-100-sell,-0.000092
1023029,2025-05-18 09:30:00+00:00,value-slippage-uniswap-v3-ethereum-dai-100-sell,-0.000091
1023030,2025-05-18 09:31:00+00:00,value-slippage-uniswap-v3-ethereum-dai-100-sell,-0.000091


In [114]:
# Pivot – szeroki DataFrame, każda kolumna to konkretny feed (buy lub sell)
slippage_wide = df_slippage.pivot(index='time', columns='feed_name', values='value')

# Lista wszystkich kolumn BUY i SELL
buy_cols = [col for col in slippage_wide.columns if col.endswith('buy')]
sell_cols = [col for col in slippage_wide.columns if col.endswith('sell')]

# 1. Metryki mean_abs_slip
mean_abs_slip_buy = slippage_wide[buy_cols].abs().mean().sort_values()
mean_abs_slip_sell = slippage_wide[sell_cols].abs().mean().sort_values()


print("Mean absolute slippage BUY (lower is better):")
print(mean_abs_slip_buy)

print("Mean absolute slippage SELL (lower is better):")
print(mean_abs_slip_sell)

Mean absolute slippage BUY (lower is better):
feed_name
value-slippage-curve-ethereum-usdt-buy            0.000043
value-slippage-uniswap-v3-ethereum-dai-100-buy    0.000044
value-slippage-curve-ethereum-dai-buy             0.000048
value-slippage-balancer-ethereum-usdt-buy         0.032099
dtype: float64
Mean absolute slippage SELL (lower is better):
feed_name
value-slippage-curve-ethereum-dai-sell             0.000003
value-slippage-curve-ethereum-usdt-sell            0.000061
value-slippage-uniswap-v3-ethereum-dai-100-sell    0.000082
value-slippage-balancer-ethereum-usdt-sell         0.032711
dtype: float64


The slippage analysis shows that feeds from Curve pools (curve-ethereum-usdt and curve-ethereum-dai) consistently offer the lowest average slippage for both buy and sell transactions, indicating high market liquidity and robust resistance to price manipulation. This makes Curve-based slippage feeds both cost-effective for users and highly reliable as pricing sources for oracles. In contrast, Balancer pools exhibit much higher slippage, meaning their reported prices can be more easily manipulated and may not accurately reflect fair market value—making them less suitable as primary feeds for USDC pricing.









In [115]:
# 2. Metryki dla SELL

std_slip_buy = slippage_wide[buy_cols].std().sort_values()
std_slip_sell = slippage_wide[sell_cols].std().sort_values()

print("\nStandard deviation of slippage BUY (lower is better):")
print(std_slip_buy)
print("\nStandard deviation of slippage SELL (lower is better):")
print(std_slip_sell)


Standard deviation of slippage BUY (lower is better):
feed_name
value-slippage-uniswap-v3-ethereum-dai-100-buy    0.000028
value-slippage-curve-ethereum-usdt-buy            0.000050
value-slippage-curve-ethereum-dai-buy             0.000054
value-slippage-balancer-ethereum-usdt-buy         0.023976
dtype: float64

Standard deviation of slippage SELL (lower is better):
feed_name
value-slippage-curve-ethereum-dai-sell             0.000001
value-slippage-uniswap-v3-ethereum-dai-100-sell    0.000034
value-slippage-curve-ethereum-usdt-sell            0.000057
value-slippage-balancer-ethereum-usdt-sell         0.024837
dtype: float64


For BUY slippage, uniswap-v3-ethereum-dai-100-buy (0.000028), curve-ethereum-usdt-buy (0.000050), and curve-ethereum-dai-buy (0.000054) have very low standard deviation, meaning their transaction costs are both low and stable. Balancer again stands out with extremely high variability (0.0240), indicating unpredictable and volatile slippage.

For SELL slippage, curve-ethereum-dai-sell (0.000001) is exceptionally stable, while uniswap-v3-ethereum-dai-100-sell (0.000034) and curve-ethereum-usdt-sell (0.000057) are also quite stable. Balancer’s standard deviation is again much higher (0.0248), signaling erratic, potentially risky slippage.

## Slippeg final

**value-slippage-balancer-ethereum-usdt** is definetly the worst one for both BUY & SELL -> worth considering removing it from final median calcualtion

**Curve and Uniswap** provide the best, most stable slippage for both buying and selling, making them ideal for price feeds and trading

-----------------------------






----------------

# Final metrics comparison

Finding feeds that have the worst metrics

### Min & Max prices
1. value-kaiko-v2 min: 0.989057 - lowest global min
2. value-coingecko	min: 0.999587	max: 1.000000 - highest global low & max never moved above 1
3. value-balancer-ethereum-usdt	max: 1.001450 (the highest max)

### GAPS

1. value-kaiko-v2 total: 9395 pct_total: 21.75%
2. value-lbank-usdt total: 1492 pct_total:3.45%

### Standard Deviation
1. value-kaiko-v2 STD: 0.000455
2. value-bitrue-usdt STD: 0.000172

### IQR Outliers

1. value-kaiko-v2                       12.43
2. value-bitrue-usdt                    10.94
3. value-bitstamp-usd                    7.50

### Flatness (feed's median movements agains global median)
**High flatness_ratio**
1. value-bitstamp-usd ratio: 2.560868
2. value-coingecko ratio: 1.988871

**Low flatness_ratio**
1. value-kaiko-v2 ratio: 0.075014
2. value-bitrue-usdt ratio: 0.505233

### Mean absolute deviation from global median
1. value-kaiko-v2                       0.000173
2. value-kucoin-usdt                    0.000093

### Feed's delay against global median

**Feeds with delays:**

1. value-bitrue-usdt (1 day count): 9 (2 days count): 2   
2. value-kaiko-v2    (1 day count): 7 (2 days count): 1   
3. value-kucoin-usdt (1 day count): 7 (2 days count): 2

**Feeds that didn't move back:**

1. value-bitstamp-usd (unresolved copunt): 113  
2. value-coingecko (unresolved copunt): 113  
3. value-uniswap-v3-ethereum-dai-100 (unresolved copunt): 60

### Lowest corellation against global median

1. value-kaiko-v2                       0.209167
2. value-bitstamp-usd                   0.285659
3. value-coingecko                      0.397027



### Critical threshold (below 0.995)

value-kaiko-v2 times: 4

rest: 0

### Critical threshold (above 1.003)

value-kaiko-v2 times: 54

rest: 0

### Slippeg:

value-slippage-balancer-ethereum is the worst

Final thoughts:
1. The worst source is definetly **coingecko**, it consistently performs the worst across all metrics.
2. **kaiko-v2** exhibits the highest volatility and has highest mean deviation from global median.
3. **value-bitstamp-usd** did not reflect the market price—similarly to coingecko—and its price movement is quite flat.
4. **value-balancer-ethereum-usdt** has the highest maximum price and the worst slippage.


It is clear that **coingecko and kaiko-v2 (and maybe value-bitstamp-usd)** should not be used to calculate the global median, while **value-balancer-ethereum-usdt** might still be considered—though it would be wise to add additional restrictions to this feed or lower its weight, if possible.